我们提供了复现ct-eQTL中单细胞数据预处理结果所需的R代码。

We provide the R script needed to reproduce the processed scRNA-seq datasets results of ct-eQTL.

In [1]:
# 载入需要的R包
# Load the required packages
library(Seurat)
library(sceasy)
library(data.table)
library(readxl)
library(tidyverse)
library(anndata)
library(SeuratDisk)

Warning message:
"package 'Seurat' was built under R version 4.0.5"
Attaching SeuratObject

Loading required package: reticulate

Warning message:
"package 'reticulate' was built under R version 4.0.5"
Warning message:
"package 'data.table' was built under R version 4.0.5"
Warning message:
"package 'tidyverse' was built under R version 4.0.5"
Registered S3 method overwritten by 'cli':
  method     from         
  print.boxx spatstat.geom

-- Attaching packages ------------------------------------------------------------------------------- tidyverse 1.3.1 --

v ggplot2 3.3.5     v purrr   0.3.4
v tibble  3.1.6     v dplyr   1.0.7
v tidyr   1.1.4     v stringr 1.4.0
v readr   2.1.1     v forcats 0.5.1

Warning message:
"package 'ggplot2' was built under R version 4.0.5"
Warning message:
"package 'tibble' was built under R version 4.0.5"
Warning message:
"package 'tidyr' was built under R version 4.0.5"
Warning message:
"package 'readr' was built under R version 4.0.5"
Warning message:
"p

In [3]:
# dataset: GSE161332
# tissue: Leaf
# Species: Arabidopsis thaliana
# file_type: txt; 10x

In [6]:
source('RNAAnnotateCelltype.r') # RNAAnnotateCelltype function from MAESTRO R package

In [7]:
# processing of GSE121619 dataset
dataset <- 'GSE161332' # 设置要处理的数据集如：GSE161332
# tissue type: Leaf
tissue <- 'Leaf' # 设置组织来源如：叶
# Species: Arabidopsis thaliana
Species <- 'Arabidopsis thaliana' # 设置物种如：拟南芥
# file_type: txt; 10x
file_type <- '10x' # 设置要处理的数据集的格式如：根据GEO下载数据的格式有txt和10X的格式，可以设置txt或10x

In [8]:
# load dataset function
read_data <- function(dataset,tissue,Species,file_type){
    if(file_type=='10x'){
        files <- list.dirs(path=dataset,recursive = F)
        datasets <- lapply(files,function(f){
            fn <- paste0(f,'/filtered_feature_bc_matrix')
            print(f)
            f <- unlist(strsplit(f,'/'))[2]
            # Setup the Seurat objects
            expr <- Read10X(fn)
            obj <- CreateSeuratObject(counts = expr, project = f, min.cells = 3, min.features = 200) # 构建Seurat对象
            obj <- subset(obj, subset = nFeature_RNA > 200 & nCount_RNA > 1000)
            # SCTransform
            obj <- SCTransform(obj, verbose = FALSE)
            return(obj)
        })
    }else if(file_type=='txt'){
        files <- list.files(dataset,pattern='txt',recursive = T)
        datasets <- lapply(files,function(f){
            fn <- paste0(dataset,'/',f)
            f <- gsub('.txt','',f)
            expr <- fread(fn)
            expr <- data.frame(expr,check.names = F)
            rownames(expr) <- expr$GENE
            expr <- expr[,-1]
            # Setup the Seurat objects
            obj <- CreateSeuratObject(counts = expr, project = f, min.cells = 3, min.features = 200) # 构建Seurat对象
            obj <- subset(obj, subset = nFeature_RNA > 200 & nCount_RNA > 1000)
            # SCTransform
            obj <- SCTransform(obj, verbose = FALSE)    
            return(obj)
        })
    }
    return(datasets)
}

In [ ]:
# load dataset
datasets <- read_data(dataset,tissue,Species,file_type)

In [10]:
# 数据整合
if(length(datasets)>1){
    # # select features that are repeatedly variable across datasets for integration
    features <- SelectIntegrationFeatures(object.list = datasets, nfeatures = 8000)
    # Run the PrepSCTIntegration() function prior to identifying anchors
    datasets <- PrepSCTIntegration(object.list = datasets, anchor.features = features, verbose = TRUE)
    datasets <- lapply(X = datasets, FUN = RunPCA, verbose = FALSE, features = features)
    # Integration
    anchors <- FindIntegrationAnchors(object.list = datasets, normalization.method = "SCT",
                                         anchor.features = features, verbose = TRUE, reference=1,reduction = "cca")
    objs <- IntegrateData(anchorset = anchors, normalization.method = "SCT", verbose = TRUE)
    rm(features,datasets,anchors)
}else{
    objs <- datasets[[1]]
    rm(datasets)
}

In [11]:
# Run PCA
objs <- RunPCA(objs, verbose = FALSE, approx = FALSE, npcs = 50)

In [12]:
# Run UMAP
objs <- RunUMAP(objs, reduction = "pca", dims = 1:50, umap.method = "umap-learn", metric = "correlation")

In [13]:
# Run TSNE
objs <- RunTSNE(objs, reduction = "pca",dims = 1:50,tsne.method = "Rtsne")

In [14]:
# Find nearest neighbors
objs <- FindNeighbors(objs, reduction = "pca",dims = 1:50)

Computing nearest neighbor graph

Computing SNN



In [15]:
#  Find clusters
objs <- FindClusters(objs, resolution = 1, algorithm = 2)

Modularity Optimizer version 1.3.0 by Ludo Waltman and Nees Jan van Eck

Number of nodes: 5186
Number of edges: 175257

Running Louvain algorithm with multilevel refinement...
Maximum modularity in 10 random starts: 0.8548
Number of communities: 20
Elapsed time: 0 seconds


In [16]:
dataset

[1] "GSE161332"

In [17]:
objs@project.name <- dataset

In [18]:
DefaultAssay(objs) <- 'SCT'

In [19]:
# 寻找差异基因
# differentially expressed genes
DEG <- FindAllMarkers(objs,
                        logfc.threshold=0.25,
                        min.diff.pct = 0.25,
                        max.cells.per.ident = 10000,
                        only.pos=T)

Calculating cluster 0

Calculating cluster 1

Calculating cluster 2

Calculating cluster 3

Calculating cluster 4

Calculating cluster 5

Calculating cluster 6

Calculating cluster 7

Calculating cluster 8

Calculating cluster 9

Calculating cluster 10

Calculating cluster 11

Calculating cluster 12

Calculating cluster 13

Calculating cluster 14

Calculating cluster 15

Calculating cluster 16

Calculating cluster 17

Calculating cluster 18

Calculating cluster 19



In [20]:
mark_gene <- DEG %>%
    mutate(avg_logFC=avg_log2FC) %>%
    filter(p_val_adj<0.05)

In [21]:
save(mark_gene,file=paste0(dataset,'_mark_gene.RData'))

In [22]:
Species

[1] "Arabidopsis thaliana"

In [23]:
# 下载已知的标记基因做细胞注释
# cell type annotation using known marker genes from plantscrnadb (http://ibi.zju.edu.cn/plantscrnadb/download.php)
if(Species=='Arabidopsis thaliana'){
    signature <- readxl::read_excel('../ath_doi_202104.xlsx')
}else if(Species=='Zea mays'){
    signature <- readxl::read_excel('../zma_doi_202104.xlsx')
    signature$Tissue[grepl('Shoot apical',signature$Tissue)] <- 'Shoot'
}else if(Species=='Oryza sativa'){
    signature <- readxl::read_excel('../osa_doi_202104.xlsx')
}

In [24]:
# signature gene 
sig_gene <- signature %>%
    as.data.frame() %>%
    filter(Tissue==tissue) %>%
    mutate(V1=`Cell Type`,V2=Cell_Marker) %>%
    unique(.) %>%
    select(V1,V2)

In [36]:
# cell type annotation by RNAAnnotateCelltype function from MAESTRO R package.
objs <- RNAAnnotateCelltype(objs, mark_gene, sig_gene, min.score = 0)

In [37]:
# meta information
meta <- read.csv(paste0("meta/",dataset,"_metaInfo.csv"))
objs@meta.data$Cells <- rownames(objs@meta.data)

In [38]:
objs@meta.data <- merge(objs@meta.data, meta, by.x="orig.ident",by.y="geo_accession",all.x = TRUE)
rownames(objs@meta.data) <- objs@meta.data$Cells 

In [40]:
# save a single R object
saveRDS(objs, file = paste0(dataset,'.rds'))